In [1]:
import os
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec

In [2]:
import numpy as np
import cv2
import os


import numpy as np

def convert_image_to_mask(rgb_mask):
    mask1 = rgb_mask.copy()  # Create a copy of the input mask
    # Perform element-wise comparison using np.all()
    mask1[np.all(mask1 == [0, 0, 0], axis=-1)] = [199, 199, 199]
    mask1[np.all(mask1 == [255, 255, 255], axis=-1)] = [255, 127,14]
    mask1[np.all(mask1 == [127, 127, 127], axis=-1)] = [31, 119, 180]
    
    return mask1

In [67]:
def create_image_grid(cases, masks, index ,save_path=None):
    # Load images and masks
    image_paths=[]
    mask_paths = []
    for key in cases.keys():
        image_paths.append( cases[key][index])
        mask_paths.append( masks[key][index])

    images = [cv2.imread(image_path) for image_path in image_paths]
    masks = [cv2.imread(mask_path) for mask_path in mask_paths]
    
    # Define number of rows and columns for the grid
    n_rows = 2
    n_cols = len(cases)
    
    # Increase figure width to accommodate 9 columns
    fig, axs = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(1 * n_cols, 2 * n_rows))
    
    # Adjust the spacing of the subplots to allow space for the bottom labels
    plt.subplots_adjust(left=0.02, right=0.98, bottom=0.15, top=0.85, wspace=0.02, hspace=0.1)


    for i, (image, mask, key) in enumerate(zip(images, masks, cases.keys())):
        #fig2, axs2 = plt.subplots(nrows=2, ncols=1, figsize=(4, 4))
        #fig2.tight_layout(w_pad=1)
        # Create a vertical plot with image and mask
        if i == 0:
            white_image = np.ones_like(images[i]) * 255
            axs[0, i].imshow(cv2.cvtColor(white_image, cv2.COLOR_BGR2RGB))
            #axs[0, i].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            axs[0, i].axis('off')
            mask = convert_image_to_mask(mask)
            axs[1, i].imshow(mask, cmap='gray')
            axs[1, i].axis('off')
            axs[1, i].set_title(key, fontsize=8, y=-0.2)

        else:
            axs[0, i].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            axs[0, i].axis('off')

            axs[1, i].imshow(cv2.cvtColor(mask, cv2.COLOR_BGR2RGB))
            axs[1, i].axis('off')
            axs[1, i].set_title(key, fontsize=7, y=-0.2)
        #fig2.title( f"{key}", fontsize=10)

    # Concatenate all columns in a plot
    #plt.subplots_adjust(wspace=0.1, hspace=0)
    plt.subplots_adjust(wspace=.05, hspace=.05)
    # Add title to the main subplot
    grid_title = os.path.basename(image_paths[-1])
    #fig.suptitle(grid_title, fontsize=12)
    if save_path is not None:
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        plt.savefig(os.path.join(save_path, grid_title))
        #plt.show()
    else:
        plt.show()
    plt.close(fig)

In [68]:
# Read the JSON config file
# Define the path to the config file
config_file = '/home/fatbardhf/SuperResolution_for_in-field_weed_images/configs/create_grid_of_images_filtering.json'
with open(config_file, 'r') as file:
    config = json.load(file)

# Extract directory paths from the config
original_dir = config["valid_images_dir"]
main_save_dir = config['save_dir']  # Directory to save the image grids
scaling_factors = config['scaling_factors']

# Get list of image files from the original directory
original_images = sorted(os.listdir( os.path.join(original_dir, "images")))
original_masks = sorted(os.listdir( os.path.join(original_dir, "masks")))

case_path = os.path.join(original_dir, "images")
mask_path = os.path.join(original_dir, "masks")

case_files = sorted(os.listdir(case_path))
mask_files = sorted(os.listdir(mask_path))

original_images = [os.path.join(case_path, filename) for filename in case_files]
original_masks = [os.path.join(mask_path, filename) for filename in mask_files]


# Get list of image files from the original directory
x1_dir = config["results"][0]["case_path"]
x1_images = sorted(os.listdir( os.path.join(x1_dir, "1x","images")))
x1_masks = sorted(os.listdir( os.path.join(x1_dir,"1x" ,"segmentations")))

x1_case_path = os.path.join(x1_dir, "1x", "images")
x1_mask_path = os.path.join(x1_dir, "1x", "segmentations")

x1_case_files = sorted(os.listdir(x1_case_path))
x1_mask_files = sorted(os.listdir(x1_mask_path))

x1_images = [os.path.join(x1_case_path, filename) for filename in x1_case_files]
x1_masks = [os.path.join(x1_mask_path, filename) for filename in x1_mask_files]


for scaling_factor in scaling_factors:
    save_dir = os.path.join(main_save_dir, scaling_factor)
    cases = {}
    masks = {}
    cases["Ground truth"] = original_images
    masks["Ground truth"] = original_masks

    cases["Original"] = x1_images
    masks["Original"] = x1_masks

    for result_case in config["results"]:
        case_path = os.path.join(result_case["case_path"], scaling_factor, "images")
        mask_path = os.path.join(result_case["case_path"], scaling_factor, "segmentations")

        case_files = sorted(os.listdir(case_path))
        mask_files = sorted(os.listdir(mask_path))

        cases[result_case["case_name"]] = [os.path.join(case_path, filename) for filename in case_files]
        masks[result_case["case_name"]] = [os.path.join(mask_path, filename) for filename in mask_files]

    # print(masks)
    for i in range(len(case_files)):
        create_image_grid(cases, masks, i, save_path=os.path.join( config["save_dir"], scaling_factor))

        

    
        
